# Ref: 
https://analyticsindiamag.com/gluonts-pytorchts-for-time-series-forecasting/

# Plot 

In [ ]:
from fund_price_loader import NAV_DIR
from fund_price_loader import load_dataset

In [ ]:
import os
from gluonts.dataset.util import to_pandas


nav_files = os.listdir(NAV_DIR)
file_path = os.path.join(NAV_DIR, nav_files[800])
print(f"file_path: {file_path}")
dataset = load_dataset(file_path)
to_pandas(list(dataset)[0]).plot(linewidth=2)

In [ ]:
from gluonts.model import prophet
predictor = prophet.ProphetPredictor(
    freq="D", prediction_length=100)
predictions = predictor.predict(dataset)
for entry, forecast in zip(dataset, predictions):
    to_pandas(entry).plot(linewidth=2)
    forecast.plot(color='g', prediction_intervals=[50.0, 90.0])

# Evaluation 

- split Train and Test Dataset 
- calculate RMSE

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
from fund_price_loader import NAV_DIR
from gluonts.dataset.util import to_pandas

nav_files = os.listdir(NAV_DIR)
file_path = os.path.join(NAV_DIR, nav_files[800])

In [ ]:
from fund_price_loader import load_split_dataset
from datetime import datetime
train, test = load_split_dataset(file_path, datetime(year=2021, month=5, day=30))

In [ ]:
to_pandas(list(train)[0]).plot(linewidth=2)
to_pandas(list(test)[0]).plot(linewidth=2)

In [ ]:
from evaluator import evaluation
from gluonts.model import prophet

evaluation(prophet.ProphetPredictor, train, test, verbose=True)

# Back Testing Splitting

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
from fund_price_loader import NAV_DIR
from gluonts.dataset.util import to_pandas
from gluonts.model import prophet
nav_files = os.listdir(NAV_DIR)
file_path = os.path.join(NAV_DIR, nav_files[800])

from backtesting import parallel_run

In [ ]:
ans = parallel_run(file_path, prophet.ProphetPredictor, 14, 70, verbose=True)

# ShareMemory with Billiard (not faster :()

In [ ]:
!pip install sharedmem

In [1]:
from fund_price_loader import load_nav_table
import os
from gluonts.dataset.util import to_pandas
from fund_price_loader import NAV_DIR

nav_files = os.listdir(NAV_DIR)
file_path = os.path.join(NAV_DIR, nav_files[800])
nav_table = load_nav_table(file_path)

/Volumes/GoogleDrive/我的雲端硬碟/github/gluonts_fund_price_forecast/venv/lib/python3.8/site-packages/gluonts/json.py:45: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [6]:
import pandas as pd
from datetime import datetime
class Transfer:
    @staticmethod
    def to_shm(smm, nav_table):
        start_end_date = smm.ShareableList([str(nav_table.index.min()), str(nav_table.index.max())])
        values = smm.ShareableList(nav_table.value.tolist())
        return start_end_date, values
    
    @staticmethod
    def to_process(start_end_date, values): 
        __string_to_date = lambda x: datetime.strptime(x.split(' ')[0], '%Y-%m-%d')
        start = start_end_date[0]
        end = start_end_date[1]
        idx = pd.date_range(start=__string_to_date(start),
                            end=__string_to_date(end), 
                            freq="D")
        nav_table = pd.DataFrame(idx, columns=['date'])
        nav_table['value'] = list(values)
        nav_table.set_index('date', inplace=True)
        return nav_table

In [5]:
from billiard.pool import Pool
from multiprocessing.managers import SharedMemoryManager
from functools import partial
def test_func(x, shared_objs=None):
    if shared_objs is None:
        return x + 1
    else:
        nav_table = Transfer.to_process(*shared_objs)
        return x + nav_table.iloc[0]

with SharedMemoryManager() as smm:
    shared_objs = Transfer.to_shm(smm, nav_table)
    with Pool(8) as p:
        ans_gen = p.imap(
            partial(test_func, shared_objs = shared_objs), range(10))
        ans = list(ans_gen)